In [91]:
import pandas as pd
import numpy as np

## Create train set for non norm target solar

In [92]:
def resample_and_interpolate(group):
    return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()

In [93]:
weather_df = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/weather_data/DWD_ICON-EU.csv")
solar_total = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/basic_files/solar_total_production.csv")
weather_df.sort_values(by='ref_datetime', inplace=True)
weather_df = weather_df.groupby(["valid_datetime","latitude","longitude"]).last().reset_index()
weather_df.reset_index(inplace=True)

In [94]:
# Did not leed to better models, so we didnt use it anymore

# weather_df["ref_datetime"] = pd.to_datetime(weather_df["ref_datetime"], utc=True)
# weather_df["valid_datetime"] = pd.to_datetime(weather_df["valid_datetime"], utc=True)
# weather_df["date_diff"] = (weather_df["valid_datetime"].dt.floor("D") - weather_df["ref_datetime"].dt.floor("D")).dt.days
# weather_df = weather_df[(weather_df["ref_datetime"].dt.time == pd.to_datetime("06:00:00").time()) & (weather_df["date_diff"] == 1)]
# weather_df = weather_df.sort_values('ref_datetime').groupby(['valid_datetime', 'latitude', 'longitude'], as_index=False).last()
# weather_df.drop_duplicates(inplace=True)

In [95]:
weather_df.valid_datetime = pd.to_datetime(weather_df.valid_datetime)
weather_df = weather_df.set_index(["valid_datetime","latitude","longitude"])
df_resampled = weather_df.groupby(['latitude', 'longitude'], group_keys=False).apply(resample_and_interpolate)
df_resampled = df_resampled.reset_index()

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_79057/2834871732.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_79057/2834871732.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_79057/2834871732.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_79057/2834871732.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return group.reset_index(level

In [97]:
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)
#df_resampled.drop(columns=['index','ref_datetime'], inplace=True)
#df_resampled.drop(columns= ["index"], inplace=True)
df_resampled_merged = pd.merge(df_resampled, solar_total, how='left', left_on='valid_datetime', right_on='timestamp_utc')
df_resampled_merged_solar = df_resampled_merged.loc[~(df_resampled_merged.latitude == 53.935) & ~(df_resampled_merged.longitude == 1.8645)]
df_resampled_merged_solar.ref_datetime = pd.to_datetime(df_resampled_merged_solar.ref_datetime)

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_79057/406553048.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resampled_merged_solar.ref_datetime = pd.to_datetime(df_resampled_merged_solar.ref_datetime)


In [99]:
df_resampled_merged_solar1 = df_resampled_merged_solar.groupby("valid_datetime").mean().reset_index()

In [100]:
distinct_lat_lon_pairs = df_resampled_merged_solar[['latitude', 'longitude']].drop_duplicates()

In [101]:
df_resampled_merged_solar1.head()

,valid_datetime,latitude,longitude,ref_datetime,Temperature,WindSpeed,WindSpeed:100,WindDirection:100,CloudCover,RelativeHumidity,PressureReducedMSL,SolarDownwardRadiation,TotalPrecipitation,date_diff,timestamp_utc,generation_mw,installed_capacity_mwp,capacity_mwp
0,2024-10-08 00:00:00+00:00,52.381681,0.650333,2024-10-07 06:00:00+00:00,13.57700,3.4225,6.83800,146.3885,1.0000,93.64350,99460.53500,0.08000,0.96450,1.0,2024-10-08 00:00:00+00:00,0.0,2956.745251,2779.477632
1,2024-10-08 00:30:00+00:00,52.381681,0.650333,2024-10-07 06:00:00+00:00,13.59850,3.3540,6.60825,150.0255,0.9870,94.02025,99449.34400,0.08525,0.62775,1.0,2024-10-08 00:30:00+00:00,0.0,2956.745251,2779.477622
2,2024-10-08 01:00:00+00:00,52.381681,0.650333,2024-10-07 06:00:00+00:00,13.62000,3.2855,6.37850,153.6625,0.9740,94.39700,99438.15300,0.09050,0.29100,1.0,2024-10-08 01:00:00+00:00,0.0,2956.745251,2779.477610
3,2024-10-08 01:30:00+00:00,52.381681,0.650333,2024-10-07 06:00:00+00:00,13.58125,3.2645,6.41575,155.7280,0.9315,94.74125,99408.31075,0.11225,0.16525,1.0,2024-10-08 01:30:00+00:00,0.0,2956.745251,2779.477610
4,2024-10-08 02:00:00+00:00,52.381681,0.650333,2024-10-07 06:00:00+00:00,13.54250,3.2435,6.45300,157.7935,0.8890,95.08550,99378.46850,0.13400,0.03950,1.0,2024-10-08 02:00:00+00:00,0.0,2956.745251,2779.477587


### Features

In [102]:
def set_up_solar_features(df):
    df["cos_day"] = np.cos(2 * np.pi * df.valid_datetime.dt.dayofyear / 365)
    df["cos_hour"] = np.cos(2 * np.pi * df.valid_datetime.dt.hour / 24)
    df["SolarDownwardRadiation_Mean"] = df.SolarDownwardRadiation
    df["Temperature_dwd_Mean"] = df.Temperature
    df["Temperature_dwd_Std"] = df_resampled_merged_solar.groupby("valid_datetime").std().reset_index().Temperature
    df["SolarDownwardRadiation_RW_dwd_Mean_30min"] = df.SolarDownwardRadiation_Mean.rolling(window=1, min_periods=1).mean()
    df["SolarDownwardRadiation_RW_Mean_1h"] = df.SolarDownwardRadiation_Mean.rolling(window=2, min_periods=1).mean()
    df["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df.SolarDownwardRadiation_Mean.shift(1)
    df["SolarDownwardRadiation_Mean_Lag_1h"] = df.SolarDownwardRadiation_Mean.shift(2)
    df["SolarDownwardRadiation_Mean_Lag_24h"] = df.SolarDownwardRadiation_Mean.shift(48)
    for i in range(len(distinct_lat_lon_pairs)):
        lat = distinct_lat_lon_pairs.latitude.iloc[i]
        lon = distinct_lat_lon_pairs.longitude.iloc[i]
        mask = (df_resampled_merged_solar.latitude == lat) & (df_resampled_merged_solar.longitude == lon)
        df[f"Temperature_{i}"] = pd.Series(df_resampled_merged_solar.Temperature[mask].values)[:len(df)]  # Fill gaps with NaN
        df[f"SolarDownwardRadiation_{i}"] = pd.Series(df_resampled_merged_solar.SolarDownwardRadiation[mask].values)[:len(df)]  # Fill gaps with NaN
    return df
df_resampled_merged_solar2 = set_up_solar_features(df_resampled_merged_solar1)

In [103]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [104]:
for i in range(20):
    temp_col = f'Temperature_{i}'
    irradiance_col = f'SolarDownwardRadiation_{i}'
    panel_temp_col = f'Panel_Temperature_Point{i}'
    panel_eff_col = f'Panel_Efficiency_Point{i}'
    df_resampled_merged_solar2[panel_temp_col], df_resampled_merged_solar2[panel_eff_col] = pv_temperature_efficiency(df_resampled_merged_solar2[irradiance_col], df_resampled_merged_solar2[temp_col])

In [105]:
df_resampled_merged_solar2["Panel_Temperature_Mean"] = df_resampled_merged_solar2.filter(regex= r"Panel_Temperature.*").mean(axis= 1)
df_resampled_merged_solar2["Panel_Efficiency_Mean"] = df_resampled_merged_solar2.filter(regex= r"Panel_Efficiency.*").mean(axis= 1)
df_resampled_merged_solar2["Panel_Temperature_Std"] = df_resampled_merged_solar2.filter(regex= r"Panel_Temperature.*").std(axis= 1)
df_resampled_merged_solar2["Panel_Efficiency_Std"] = df_resampled_merged_solar2.filter(regex= r"Panel_Efficiency.*").std(axis= 1)

In [106]:
df_resampled_merged_solar2["Solar_MWh_Lag_48h"] = df_resampled_merged_solar2.generation_mw.shift(periods= 96) * 0.5
df_resampled_merged_solar2["Solar_MWh_credit"] = df_resampled_merged_solar2.generation_mw * 0.5
df_resampled_merged_solar2["Capacity_MWP_Lag_48h"] = df_resampled_merged_solar2.capacity_mwp.shift(periods= 96)

In [109]:
df_resampled_merged_solar3 = df_resampled_merged_solar2[[ 
    "SolarDownwardRadiation_Mean",
    "SolarDownwardRadiation_RW_Mean_1h",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_Mean_Lag_1h",
    "SolarDownwardRadiation_Mean_Lag_24h",
    "Panel_Efficiency_Mean",
    "Panel_Efficiency_Std",
    "Panel_Temperature_Mean",
    "Panel_Temperature_Std",
    "Temperature_dwd_Std",
    "Temperature_dwd_Mean",
    "cos_hour",
    "cos_day",
    "Capacity_MWP_Lag_48h",
    "Solar_MWh_Lag_48h",
    "Solar_MWh_credit"
    ]]
df_resampled_merged_solar3.dropna(inplace=True)

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_79057/2103203449.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resampled_merged_solar3.dropna(inplace=True)


In [110]:
df_resampled_merged_solar3.columns

Index(['SolarDownwardRadiation_Mean', 'SolarDownwardRadiation_RW_Mean_1h',
       'SolarDownwardRadiation_RW_dwd_Mean_30min',
       'SolarDownwardRadiation_dwd_Mean_Lag_30min',
       'SolarDownwardRadiation_Mean_Lag_1h',
       'SolarDownwardRadiation_Mean_Lag_24h', 'Panel_Efficiency_Mean',
       'Panel_Efficiency_Std', 'Panel_Temperature_Mean',
       'Panel_Temperature_Std', 'Temperature_dwd_Std', 'Temperature_dwd_Mean',
       'cos_hour', 'cos_day', 'Capacity_MWP_Lag_48h', 'Solar_MWh_Lag_48h',
       'Solar_MWh_credit'],
      dtype='object')

In [111]:
df_resampled_merged_solar3.to_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/test.csv", index=False)

## Create test set for norm target

In [112]:
import numpy as np

df_resampled_merged_solar2["Target_Capacity_MWP_%"] = np.where(df_resampled_merged_solar2["Solar_MWh_credit"] == 0.0, 0, df_resampled_merged_solar2["Solar_MWh_credit"] / df_resampled_merged_solar2["capacity_mwp"])
df_resampled_merged_solar2["Target_Capacity_MWP_%_Lag_48"] = df_resampled_merged_solar2["Target_Capacity_MWP_%"].shift(96)

In [113]:
df_resampled_merged_solar2["capacity_mwp"].mean()

np.float64(2778.886001580911)

In [114]:
df_resampled_merged_solar4 = df_resampled_merged_solar2[[ 
    "SolarDownwardRadiation_Mean",
    "SolarDownwardRadiation_RW_Mean_1h",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_Mean_Lag_1h",
    "SolarDownwardRadiation_Mean_Lag_24h",
    "Panel_Efficiency_Mean",
    "Panel_Efficiency_Std",
    "Panel_Temperature_Mean",
    "Panel_Temperature_Std",
    "Temperature_dwd_Std",
    "Temperature_dwd_Mean",
    "cos_hour",
    "cos_day",
    "Solar_MWh_Lag_48h",
    "Capacity_MWP_Lag_48h",
    "Target_Capacity_MWP_%_Lag_48",
    "Target_Capacity_MWP_%",
    ]]
df_resampled_merged_solar4.dropna(inplace=True)

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_79057/1790439630.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resampled_merged_solar4.dropna(inplace=True)


In [115]:
df_resampled_merged_solar4.columns

,SolarDownwardRadiation_Mean,SolarDownwardRadiation_RW_Mean_1h,SolarDownwardRadiation_RW_dwd_Mean_30min,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_Mean_Lag_1h,SolarDownwardRadiation_Mean_Lag_24h,Panel_Efficiency_Mean,Panel_Efficiency_Std,Panel_Temperature_Mean,Panel_Temperature_Std,Temperature_dwd_Std,Temperature_dwd_Mean,cos_hour,cos_day,Solar_MWh_Lag_48h,Capacity_MWP_Lag_48h,Target_Capacity_MWP_%_Lag_48,Target_Capacity_MWP_%
96,0.12250,0.153625,0.12250,0.18475,0.24700,0.22900,0.190409,0.000495,10.543578,0.687418,0.380345,11.66100,1.000000,0.175531,0.0,2779.477632,0.0,0.0
97,0.16350,0.143000,0.16350,0.12250,0.18475,0.16025,0.190469,0.000474,10.459594,0.658849,0.428622,11.50475,1.000000,0.175531,0.0,2779.477622,0.0,0.0
98,0.20450,0.184000,0.20450,0.16350,0.12250,0.09150,0.190469,0.000474,10.459594,0.658849,0.496782,11.34850,0.965926,0.175531,0.0,2779.477610,0.0,0.0
99,0.17325,0.188875,0.17325,0.20450,0.16350,0.11350,0.190446,0.000481,10.491406,0.668183,0.564547,11.12350,0.965926,0.175531,0.0,2779.477610,0.0,0.0
100,0.14200,0.157625,0.14200,0.17325,0.20450,0.13550,0.190446,0.000481,10.491406,0.668183,0.637935,10.89850,0.866025,0.175531,0.0,2779.477587,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,0.30900,0.345125,0.30900,0.38125,0.45350,0.43300,0.189716,0.000357,11.505766,0.495302,0.493878,8.58000,0.707107,0.452072,0.0,2778.376631,0.0,0.0
919,0.31325,0.311125,0.31325,0.30900,0.38125,0.40175,0.189716,0.000357,11.505766,0.495302,0.420318,8.44800,0.707107,0.452072,0.0,2778.376610,0.0,0.0
920,0.31750,0.315375,0.31750,0.31325,0.30900,0.37050,0.189731,0.000395,11.484492,0.549218,0.379715,8.31600,0.500000,0.452072,0.0,2778.376599,0.0,0.0
921,0.35750,0.337500,0.35750,0.31750,0.31325,0.37150,0.189731,0.000395,11.484492,0.549218,0.352959,8.19275,0.500000,0.452072,0.0,2778.376598,0.0,0.0


In [116]:
df_api = df_resampled_merged_solar4.copy()

In [117]:
df_resampled_merged_solar4.to_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/test_norm1.csv", index=False)